In [1]:
# pip install selenium beautifulsoup4 pandas mysql-connector-python

In [2]:
# pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
import mysql.connector

In [4]:
chrome_path = "./chromedriver.exe"
service = Service(executable_path=chrome_path)
driver = webdriver.Chrome(service=service)

driver.get("https://www.google.com")
print("크롬 실행 성공!")
driver.quit()

크롬 실행 성공!


In [5]:
# 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창 없이 실행
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# 드라이버 경로 설정 (수정 필요)
service = Service('./chromedriver.exe')  # 여기 경로로 바꿔야됨.
driver = webdriver.Chrome(service=service, options=chrome_options)

# 검색어
keyword = '연두색 번호판'
# 네이버 뉴스 검색 URL
url = f"https://search.naver.com/search.naver?where=news&query={keyword}"
driver.get(url)
time.sleep(3)  # 로딩 대기

# 페이지 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 뉴스 기사 박스 선택
articles = soup.select("div.news_area")

if not articles:
    print("❗ 뉴스 기사를 찾을 수 없습니다.")
else:
    for article in articles:
        title_tag = article.select_one("a.news_tit")
        press_tag = article.select_one("a.info.press")
        date_tag = article.select_one("span.info")

        title = title_tag["title"].strip() if title_tag else "제목 없음"
        link = title_tag["href"] if title_tag else "링크 없음"
        press = press_tag.text.strip() if press_tag else "언론사 없음"
        date = date_tag.text.strip() if date_tag else "날짜 없음"

        print(f"제목: {title}")
        print(f"링크: {link}")
        print(f"언론사: {press}")
        print(f"날짜: {date}")
        print("-" * 50)

driver.quit()


제목: 연두색 번호판의 기습…S클래스 독주 끝? [데이터로 보는 세상]
링크: https://www.mk.co.kr/article/11276278
언론사: 매경이코노미
날짜: 3일 전
--------------------------------------------------
제목: '연두색 번호판' 기피 현상 감소…고급 수입차 판매 '꿈틀'
링크: https://www.newsis.com/view/NISX20250324_0003110599
언론사: 뉴시스언론사 선정
날짜: 2주 전
--------------------------------------------------
제목: 번호판도 싫고, 살 차도 없고… 고가 수입차 '내리막'
링크: https://www.dailian.co.kr/news/view/1477573/?sc=Naver
언론사: 데일리안언론사 선정
날짜: 1주 전
--------------------------------------------------
제목: "정체된 S클래스"···벤츠, 더 위 노린다
링크: https://www.sisajournal-e.com/news/articleView.html?idxno=410659
언론사: 시사저널이코노미
날짜: 6일 전
--------------------------------------------------
제목: 연두색 번호판 기피 현상 감소? 올해 들어 고급 수입차 판매 늘어
링크: http://www.wolyo.co.kr/news/articleView.html?idxno=257386
언론사: 월요신문
날짜: 1주 전
--------------------------------------------------
제목: '맞수' 벤츠 S클래스·BMW 7시리즈…'영원한 왕좌'는 없다?
링크: https://www.gpkorea.com/news/articleView.html?idxno=126703
언론사: 지피코리아
날짜: 1주 전
----------------------------------------

In [6]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 🧩 기사 본문 추출 함수
def extract_article_content(driver, link):
    try:
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        if "mk.co.kr" in link:
            content = soup.select_one("div.art_txt")
        elif "newsis.com" in link:
            content = soup.select_one("div#textBody") or soup.select_one("div.view_con_tx")
        elif "dailian.co.kr" in link:
            content = soup.select_one("div.view-content") or soup.select_one("div#viewContent")
        elif any(domain in link for domain in ["sisajournal-e.com", "wolyo.co.kr", "gpkorea.com", "hansbiz.co.kr"]):
            content = soup.select_one("div#article-view-content-div")
        elif "chosun.com" in link:
            content = soup.select_one("div#news_article")
        elif "segye.com" in link:
            content = soup.select_one("div#article_txt")
        else:
            content = None

        return content.get_text(separator=" ", strip=True) if content else "본문 추출 실패"

    except Exception as e:
        print(f"[에러] {link} 본문 추출 실패: {e}")
        return "본문 추출 오류"

# 🧰 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# 드라이버 경로 수정 필요
service = Service('./chromedriver.exe')  # 👉 여기를 본인 PC에 맞게 수정
driver = webdriver.Chrome(service=service, options=chrome_options)

# 🔍 검색어 설정
keyword = "연두색 번호판"
search_url = f"https://search.naver.com/search.naver?where=news&query={keyword}"
driver.get(search_url)
time.sleep(3)

# 페이지 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")
articles = soup.select("div.news_area")

results = []

# 뉴스 제목 & 링크 수집 및 본문 추출
for article in articles:
    title_tag = article.select_one("a.news_tit")
    if not title_tag:
        continue

    title = title_tag["title"].strip()
    link = title_tag["href"]

    print(f"📄 기사 추출 중: {title}")
    content = extract_article_content(driver, link)

    results.append({
        "제목": title,
        "본문": content,
        "링크": link
    })

# 📝 CSV로 저장
csv_filename = "naver_news.csv"
with open(csv_filename, mode="w", encoding="utf-8-sig", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["제목", "본문", "링크"])
    writer.writeheader()
    writer.writerows(results)

print(f"\n✅ {len(results)}개의 기사를 '{csv_filename}'에 저장 완료!")

driver.quit()


📄 기사 추출 중: 연두색 번호판의 기습…S클래스 독주 끝? [데이터로 보는 세상]
📄 기사 추출 중: '연두색 번호판' 기피 현상 감소…고급 수입차 판매 '꿈틀'
📄 기사 추출 중: 번호판도 싫고, 살 차도 없고… 고가 수입차 '내리막'
📄 기사 추출 중: "정체된 S클래스"···벤츠, 더 위 노린다
📄 기사 추출 중: 연두색 번호판 기피 현상 감소? 올해 들어 고급 수입차 판매 늘어
📄 기사 추출 중: '맞수' 벤츠 S클래스·BMW 7시리즈…'영원한 왕좌'는 없다?
📄 기사 추출 중: 연두색 번호판 탓?… 고가 수입차, 영리치 대신 50대가 사
📄 기사 추출 중: '연두색 번호판'에도 본국보다 판매 많아…4억 '테메라리오' 상륙
📄 기사 추출 중: ‘연두색 아니네’…공수처장 관용차 번호판 바꿀 수 있던 이유는
📄 기사 추출 중: “연두색 번호판 창피해서?”···수입차 시세 8년만에 ‘뚝뚝’

✅ 10개의 기사를 'naver_news.csv'에 저장 완료!


In [7]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def extract_article_content(driver, url):
    try:
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 언론사별 본문 추출 로직
        if 'mk.co.kr' in url:
            content = soup.select_one('div#article_body')
        elif 'newsis.com' in url:
            content = soup.select_one('div#textBody')
        elif 'dailian.co.kr' in url:
            content = soup.select_one('div#articleContent')
        elif 'sisajournal-e.com' in url or 'wolyo.co.kr' in url or 'gpkorea.com' in url or 'hansbiz.co.kr' in url:
            content = soup.select_one('div#article-view-content-div')
        elif 'chosun.com' in url:
            content = soup.select_one('div#news_body_id')
        elif 'segye.com' in url:
            content = soup.select_one('div.article_txt')
        else:
            print(f"❌ [{url}] 해당 언론사에 대한 본문 추출 로직이 없습니다.")
            return None

        if content:
            return content.get_text(separator=' ', strip=True)
        else:
            print(f"❌ [{url}] 본문을 찾을 수 없습니다.")
            return None

    except Exception as e:
        print(f"❌ [{url}] 본문 추출 중 에러 발생: {e}")
        return None

# 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
service = Service('./chromedriver.exe')  # chromedriver 경로 설정
driver = webdriver.Chrome(service=service, options=chrome_options)

# 테스트할 기사 URL 리스트
article_urls = [
    'https://www.newsis.com/view/NISX20250324_0003110599',
    # 다른 기사 URL 추가 가능
]

# 각 기사에 대해 본문 추출 테스트
for url in article_urls:
    print(f"🔗 기사 URL: {url}")
    content = extract_article_content(driver, url)
    if content:
        print(f"📝 본문 내용: {content[:100]}...")  # 본문 내용의 앞부분만 출력
    print("-" * 80)

driver.quit()


🔗 기사 URL: https://www.newsis.com/view/NISX20250324_0003110599
📝 본문 내용: [수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 ...
--------------------------------------------------------------------------------


In [8]:
content # 본문 크롤링을 잘못하고있음.

'[수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 정리하고 있다. 2024.01.08. jtk@newsis.com'

In [9]:
import requests
from bs4 import BeautifulSoup
# keyword = "법인차량"
url = ("https://www.newsis.com/view/NISX20250324_0003110599")

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select_one('#content > div.articleView > div.view > div.viewer > article')
    print(title)
else : 
    print(response.status_code)

<article>
<div class="summury">
<h2>'연두색 번호판' 수입 법인차 증가세<br/></h2><h2>2024년 시행 직후 연간 판매량 감소<br/></h2><h2>1~2월 BMW·벤츠 판매율 반등 조짐<br/></h2><h2>람보르기니 630%, 벤틀리 47% 증가</h2>
</div>
<div id="view_ad"></div>
<div align="justify" class="thumCont"><div id="textBody"><div class="article_photo thum center" id="imgartitable_NISI20240108_0020186673"><tbody><tr><td class="img"><img alt="[수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 정리하고 있다. 2024.01.08. jtk@newsis.com" arti_id="NISI20240108_0020186673" check_caption="Y" mode_we="edit" src="https://img1.newsis.com/2024/01/08/NISI20240108_0020186673_web.jpg?rnd=20240108135648" style="float:; margin:0 auto;display:block;"/></td></tr><tr><p class="desc" id="caption_NISI20240108_0020186673"><p class="photojournal">[수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 정리하고 있다. 2024.01.08. <a class="__cf_email__" data-cfemail="93f9e7f8d3fdf6e4e0fae0bdf0fcfe" h

In [10]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.newsis.com/view/NISX20250324_0003110599'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(response.content, 'html.parser')

# 1. 전체 기사 영역
article_body = soup.select_one('#content > div.articleView > div.view > div.viewer > article')

if article_body:
    # 2. 불필요한 태그 제거 (사진, 캡션 등)
    for tag in article_body.select('img, p.desc, p.photojournal, h2, table, script, style'):
        tag.decompose()

    # 3. 텍스트만 추출
    clean_text = article_body.get_text(separator="\n", strip=True)
    print(clean_text)
else:
    print("본문 영역을 찾을 수 없습니다.")

[서울=뉴시스]박현준 기자 = 지난해부터 시행한 '연두색 번호판' 제도로 고급 수입 법인차 판매량이 급감했지만, 올해 다시 반등 조짐을 보이고 있다.
24일 한국수입자동차협회(KAIDA)에 따르면 지난달까지 법인으로 등록한 1억원 이상 수입차는 5501대로 파악됐다. 지난해 같은 기간(4394대) 대비 25%가량 증가한 수치다.
지난해 초부터 정부는 법인 차량의 사적 사용을 막기 위해 취득가액 8000만원 이상 차량엔 의무적으로 연두색 번호판을 붙이는 정책을 시행했다.
이에 고급 수입 법인차부터 고위 '럭셔리 카' 브랜드에 이르는 차량 판매가 감소하는 현상이 나타났다.
지난 2023년 1억원 이상 수입 법인차의 연간 판매량은 5만1083대였다. 하지만 정책이 시행된 2024년 한 해 동안 3만5320대가 팔리며 연간 판매량이 30% 넘게 주저앉았다.
이 중에서도 소위 '럭셔리 카'로 분류되는 수입 브랜드의 감소 폭이 컸다. 2023년 1~2월과 지난해 같은 기간의 1억원 이상 법인 차량의 판매량을 비교했을 때 벤틀리(-83.2%), 람보르기니(-73.7%), 롤스로이스(-43.3%) 등에서 큰 감소세가 나타났다.
하지만 올해 1~2월 1억원 이상 BMW 법인 차량 판매량은 전년보다 31.8% 증가한 2296대, 메르세데스-벤츠도 15.6% 증가한 1546대로 판매율이 반등할 조짐이다.
특히 람보르기니의 경우 지난해 같은 기간 대비 630% 증가한 73대를 판매하는 등 고가 법인차 시장을 이끄는 것으로 나타났다. 벤틀리 역시 47.1% 증가한 25대를 기록했고, 롤스로이스는 전년과 비슷한 수준인 14대로 집계됐다.
업계 관계자는 "연두색 번호판을 도입한 지 1년이 넘으며 기피 현상이 점점 줄어들고 있다"며 "고급 법인차량 판매가 급감한 지난해보다 올해 판매량이 더 늘어날 것으로 예상된다"고 말했다.
◎공감언론 뉴시스
[email protected]
